# bugs crawler (total emotion)

In [4]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import re
from IPython.display import display
import pandas as pd
import datetime as dt

import os
os.chdir('/content/drive/MyDrive/포토 뮤직리스트/크롤링/data')

In [7]:
soup = BeautifulSoup(requests.get('https://music.bugs.co.kr/musicpd?wl_ref=M_left_02_04').content, 'lxml') 
dict_categorytheme={}
for a_tag in soup.select('ul.listCategoryTheme')[3].select('a.hyrend'):
    dict_categorytheme[a_tag.get_text().rstrip()]=a_tag['href'] #장르 오른쪽 공백 지우기
dict_categorytheme.keys()

dict_keys(['사랑/기쁨', '이별/슬픔', '스트레스/짜증', '우울할때', '지치고 힘들때', '멘붕/불안', '그리움', '외로울때', '썸 탈때', '고백', '울고 싶을때', '새벽감성', '싱숭생숭', '설렘/심쿵', '기분전환', '힐링'])

In [8]:
keys1 = ('사랑/기쁨', '이별/슬픔', '스트레스/짜증')
d1 = {k: dict_categorytheme[k] for k in keys1}
keys2 = ('우울할때', '지치고 힘들때', '멘붕/불안')
d2 = {k: dict_categorytheme[k] for k in keys2}
keys3 = ('그리움', '외로울때', '썸 탈때')
d3 = {k: dict_categorytheme[k] for k in keys3}
keys4 = ('고백', '울고 싶을때', '새벽감성')
d4 = {k: dict_categorytheme[k] for k in keys4}
keys5 = ('싱숭생숭', '설렘/심쿵', '기분전환', '힐링')
d5 = {k: dict_categorytheme[k] for k in keys5}

In [9]:
d1

{'사랑/기쁨': 'https://music.bugs.co.kr/musicpd?tag_id=5729',
 '스트레스/짜증': 'https://music.bugs.co.kr/musicpd?tag_id=5963',
 '이별/슬픔': 'https://music.bugs.co.kr/musicpd?tag_id=5712'}

### d1

In [12]:
dict_category = {'category_emotion':[],'artist_disp_nm':[],'track_title':[],'album_style':[]}

for name_categorytheme,url_categorytheme in d1.items():
    soup_category = BeautifulSoup(requests.get(url_categorytheme).content, 'lxml')
    page_num = int(re.sub(r'\D', '', soup_category.select_one('p.desc').get_text()))//24 + 2
    base_url = soup_category.select('div.paging > a')[0]['href'][:-1]
    album_count=0
    for n_th_page in range(1,page_num): # page_num
        url_categorypages=base_url+f'{n_th_page}'
        dict_categoryalbum={}
        soup_pages = BeautifulSoup(requests.get(url_categorypages).content, 'lxml')
        for album_tag in soup_pages.find_all('a', class_="title hyrend"):
            dict_categoryalbum[album_tag['title']]=album_tag['href']
        for url_categoryalbum in dict_categoryalbum.values():
            album_count+=1
            soup_album = BeautifulSoup(requests.get(url_categoryalbum).content, 'lxml')
            ele_album_style = str() 
            for a_tag in soup_album.select('div.tag > a.hyrend'):
                ele_album_style+=a_tag.get_text()
            for music_tag in soup_album.find_all('a',attrs={'class': 'btnActions','album_title':False}): # soup_album.select('a.btnActions')
                dict_category['category_emotion'].append(name_categorytheme)
                dict_category['artist_disp_nm'].append(music_tag['artist_disp_nm'])
                dict_category['track_title'].append(music_tag['track_title'])
                dict_category['album_style'].append(ele_album_style)
        if n_th_page%20==0:
            print(f'{n_th_page}번째 페이지',f'앨범{album_count}개')
            display(pd.DataFrame.from_dict(dict_category).tail(2))
pd.DataFrame.from_dict(dict_category).shape
data = pd.DataFrame.from_dict(dict_category)
data.to_csv('data_emotion_d1.csv', index = False, encoding = 'UTF-8')

20번째 페이지 앨범480개


,category_emotion,artist_disp_nm,track_title,album_style
9560,사랑/기쁨,지나(G.NA),애인이 생기면 하고 싶은 일 (With 비),#국내#달달한#설렘/심쿵#사랑/기쁨#고백
9561,사랑/기쁨,시크릿(Secret),별빛달빛,#국내#달달한#설렘/심쿵#사랑/기쁨#고백


40번째 페이지 앨범960개


,category_emotion,artist_disp_nm,track_title,album_style
18256,사랑/기쁨,성훈 [Brown Eyed Soul],라리라,#달달한#사랑/기쁨#화창한 날#빼빼로데이#고백
18257,사랑/기쁨,팔로(FarLaw),Romantic Comedy (로맨틱코미디) (With 고미정),#달달한#사랑/기쁨#화창한 날#빼빼로데이#고백


20번째 페이지 앨범480개


,category_emotion,artist_disp_nm,track_title,album_style
30791,이별/슬픔,박원,걸어간다,#국내#가요#발라드한#이별/슬픔#감성적인#국내 발라드
30792,이별/슬픔,손성제,Goodbye,#국내#가요#발라드한#이별/슬픔#감성적인#국내 발라드


40번째 페이지 앨범960개


,category_emotion,artist_disp_nm,track_title,album_style
39213,이별/슬픔,아이비(IVY),좋은 사람,#가요#국내#애절한#이별/슬픔
39214,이별/슬픔,별,나빠,#가요#국내#애절한#이별/슬픔


20번째 페이지 앨범480개


,category_emotion,artist_disp_nm,track_title,album_style
51241,스트레스/짜증,Mary J. Blige(메리 제이 블라이즈),Any Way You Want It,#드라이브#락#신나는#스트레스/짜증#해외
51242,스트레스/짜증,Gerard Way(제라드 웨이),Maya The Psychic,#드라이브#락#신나는#스트레스/짜증#해외


In [13]:
display(pd.DataFrame.from_dict(dict_category))

,category_emotion,artist_disp_nm,track_title,album_style
0,사랑/기쁨,BØrns(본즈),Electric Love,#해외#락#사랑/기쁨#모던락/팝락/인디락#팝송#해외 팝
1,사랑/기쁨,The Vamps(뱀프스),Somebody To You,#해외#락#사랑/기쁨#모던락/팝락/인디락#팝송#해외 팝
2,사랑/기쁨,Eden(에덴),sex,#해외#락#사랑/기쁨#모던락/팝락/인디락#팝송#해외 팝
3,사랑/기쁨,Harry Styles(해리 스타일스),Adore You,#해외#락#사랑/기쁨#모던락/팝락/인디락#팝송#해외 팝
4,사랑/기쁨,Labrinth(라브린느),Still Don't Know My Name,#해외#락#사랑/기쁨#모던락/팝락/인디락#팝송#해외 팝
...,...,...,...,...
52912,스트레스/짜증,Fenix TX(페닉스 티엑스),All My Fault,#신나는#펑크락#세부장르#스트레스/짜증#명곡콜렉션#로큰롤라이프
52913,스트레스/짜증,Bowling For Soup(볼링 포 수프),1985,#신나는#펑크락#세부장르#스트레스/짜증#명곡콜렉션#로큰롤라이프
52914,스트레스/짜증,Weezer(위저),Put Me Back Together,#신나는#펑크락#세부장르#스트레스/짜증#명곡콜렉션#로큰롤라이프
52915,스트레스/짜증,All Time Low(올 타임 로우),I Feel Like Dancin',#신나는#펑크락#세부장르#스트레스/짜증#명곡콜렉션#로큰롤라이프


### d2

In [14]:
dict_category = {'category_emotion':[],'artist_disp_nm':[],'track_title':[],'album_style':[]}

for name_categorytheme,url_categorytheme in d2.items():
    soup_category = BeautifulSoup(requests.get(url_categorytheme).content, 'lxml')
    page_num = int(re.sub(r'\D', '', soup_category.select_one('p.desc').get_text()))//24 + 2
    base_url = soup_category.select('div.paging > a')[0]['href'][:-1]
    album_count=0
    for n_th_page in range(1,page_num): # page_num
        url_categorypages=base_url+f'{n_th_page}'
        dict_categoryalbum={}
        soup_pages = BeautifulSoup(requests.get(url_categorypages).content, 'lxml')
        for album_tag in soup_pages.find_all('a', class_="title hyrend"):
            dict_categoryalbum[album_tag['title']]=album_tag['href']
        for url_categoryalbum in dict_categoryalbum.values():
            album_count+=1
            soup_album = BeautifulSoup(requests.get(url_categoryalbum).content, 'lxml')
            ele_album_style = str() 
            for a_tag in soup_album.select('div.tag > a.hyrend'):
                ele_album_style+=a_tag.get_text()
            for music_tag in soup_album.find_all('a',attrs={'class': 'btnActions','album_title':False}): # soup_album.select('a.btnActions')
                dict_category['category_emotion'].append(name_categorytheme)
                dict_category['artist_disp_nm'].append(music_tag['artist_disp_nm'])
                dict_category['track_title'].append(music_tag['track_title'])
                dict_category['album_style'].append(ele_album_style)
        if n_th_page%20==0:
            print(f'{n_th_page}번째 페이지',f'앨범{album_count}개')
            display(pd.DataFrame.from_dict(dict_category).tail(2))
pd.DataFrame.from_dict(dict_category).shape
data = pd.DataFrame.from_dict(dict_category)
data.to_csv('data_emotion_d2.csv', index = False, encoding = 'UTF-8')

20번째 페이지 앨범480개


,category_emotion,artist_disp_nm,track_title,album_style
8526,우울할때,Eco bridge,어느 날 문득,#팝송#가요#우울할때#해외 팝
8527,우울할때,Pink Martini(핑크 마티니),Hey Eugene,#팝송#가요#우울할때#해외 팝


20번째 페이지 앨범480개


,category_emotion,artist_disp_nm,track_title,album_style
21102,지치고 힘들때,멜로망스(MeloMance),무엇을 해야 할까,#국내#가요#인디#지치고 힘들때#취준/수험생#위로#하교/퇴근길#지하철/버스
21103,지치고 힘들때,E SENS,Sh All Day,#국내#가요#인디#지치고 힘들때#취준/수험생#위로#하교/퇴근길#지하철/버스


40번째 페이지 앨범960개


,category_emotion,artist_disp_nm,track_title,album_style
29631,지치고 힘들때,예상밖의 일주일,같은 생각,#가요#오후#국내#지치고 힘들때
29632,지치고 힘들때,라임어택(RHYME-A-),혼자라고 느낄 때 (Acoustic Ver),#가요#오후#국내#지치고 힘들때


In [15]:
display(pd.DataFrame.from_dict(dict_category))

,category_emotion,artist_disp_nm,track_title,album_style
0,우울할때,소니아(Sonia),행복의 주문,#국내외#뉴에이지#피아노 뉴에이지#연주곡#피아노 연주곡#피아노#우울할때
1,우울할때,데일리 스케치,5월의 봄 날,#국내외#뉴에이지#피아노 뉴에이지#연주곡#피아노 연주곡#피아노#우울할때
2,우울할때,Lewisia(레위시아),세월,#국내외#뉴에이지#피아노 뉴에이지#연주곡#피아노 연주곡#피아노#우울할때
3,우울할때,앙꼬 피아노(Anko Piano),너를 걸어,#국내외#뉴에이지#피아노 뉴에이지#연주곡#피아노 연주곡#피아노#우울할때
4,우울할때,벚꽃(Cherry Blossom),비 온 뒤,#국내외#뉴에이지#피아노 뉴에이지#연주곡#피아노 연주곡#피아노#우울할때
...,...,...,...,...
33573,멘붕/불안,Walter Lupi,A Whiter Shade Of Pale,#가을#독서#어쿠스틱한#멘붕/불안#밤/새벽#가을밤#포크/어쿠스틱#기타연주곡
33574,멘붕/불안,정성하,One Of Us,#가을#독서#어쿠스틱한#멘붕/불안#밤/새벽#가을밤#포크/어쿠스틱#기타연주곡
33575,멘붕/불안,이현석(Lee Hyun Suk),After The Rain,#가을#독서#어쿠스틱한#멘붕/불안#밤/새벽#가을밤#포크/어쿠스틱#기타연주곡
33576,멘붕/불안,김광석,숲 속 (In The Forest),#가을#독서#어쿠스틱한#멘붕/불안#밤/새벽#가을밤#포크/어쿠스틱#기타연주곡


### d3

In [17]:
import os
os.chdir('/content/drive/MyDrive/포토 뮤직리스트/크롤링/data')

In [18]:
dict_category = {'category_emotion':[],'artist_disp_nm':[],'track_title':[],'album_style':[]}

for name_categorytheme,url_categorytheme in d3.items():
    soup_category = BeautifulSoup(requests.get(url_categorytheme).content, 'lxml')
    page_num = int(re.sub(r'\D', '', soup_category.select_one('p.desc').get_text()))//24 + 2
    base_url = soup_category.select('div.paging > a')[0]['href'][:-1]
    album_count=0
    for n_th_page in range(1,page_num): # page_num
        url_categorypages=base_url+f'{n_th_page}'
        dict_categoryalbum={}
        soup_pages = BeautifulSoup(requests.get(url_categorypages).content, 'lxml')
        for album_tag in soup_pages.find_all('a', class_="title hyrend"):
            dict_categoryalbum[album_tag['title']]=album_tag['href']
        for url_categoryalbum in dict_categoryalbum.values():
            album_count+=1
            soup_album = BeautifulSoup(requests.get(url_categoryalbum).content, 'lxml')
            ele_album_style = str() 
            for a_tag in soup_album.select('div.tag > a.hyrend'):
                ele_album_style+=a_tag.get_text()
            for music_tag in soup_album.find_all('a',attrs={'class': 'btnActions','album_title':False}): # soup_album.select('a.btnActions')
                dict_category['category_emotion'].append(name_categorytheme)
                dict_category['artist_disp_nm'].append(music_tag['artist_disp_nm'])
                dict_category['track_title'].append(music_tag['track_title'])
                dict_category['album_style'].append(ele_album_style)
        if n_th_page%20==0:
            print(f'{n_th_page}번째 페이지',f'앨범{album_count}개')
            display(pd.DataFrame.from_dict(dict_category).tail(2))
pd.DataFrame.from_dict(dict_category).shape
data = pd.DataFrame.from_dict(dict_category)
data.to_csv('data_emotion_d3.csv', index = False, encoding = 'UTF-8')
display(pd.DataFrame.from_dict(dict_category))

20번째 페이지 앨범480개


,category_emotion,artist_disp_nm,track_title,album_style
8627,그리움,스탠딩 에그(Standing Egg),사랑한다는 말,#인디#그리움#국내#애절한
8628,그리움,어른아이(Adultchild),Sad Thing (드라마 '커피프린스 1호점' 삽입곡),#인디#그리움#국내#애절한


,category_emotion,artist_disp_nm,track_title,album_style
0,그리움,Conan Gray,Astronomy,#국내외#이별/슬픔#그리움#멜랑콜리#싱숭생숭#쓸쓸한#비/흐림#감성적인#짝사랑
1,그리움,Gert Taberner,Georgia,#국내외#이별/슬픔#그리움#멜랑콜리#싱숭생숭#쓸쓸한#비/흐림#감성적인#짝사랑
2,그리움,Lewis Capaldi,Before You Go,#국내외#이별/슬픔#그리움#멜랑콜리#싱숭생숭#쓸쓸한#비/흐림#감성적인#짝사랑
3,그리움,Dan & Shay(댄 & 셰이),I Should Probably Go To Bed,#국내외#이별/슬픔#그리움#멜랑콜리#싱숭생숭#쓸쓸한#비/흐림#감성적인#짝사랑
4,그리움,Ariana Grande(아리아나 그란데),pov,#국내외#이별/슬픔#그리움#멜랑콜리#싱숭생숭#쓸쓸한#비/흐림#감성적인#짝사랑
...,...,...,...,...
17643,썸 탈때,제이켠(J\'Kyun),영화봐요 우리 (feat. 오수민),#가요#인디#달달한#이색테마#썸 탈때
17644,썸 탈때,린(LYn),곰인형 (feat. 해금),#가요#인디#달달한#이색테마#썸 탈때
17645,썸 탈때,다이나믹 듀오,해뜰때까지만 (Girl),#가요#인디#달달한#이색테마#썸 탈때
17646,썸 탈때,범키,갖고놀래 (feat. 다이나믹 듀오),#가요#인디#달달한#이색테마#썸 탈때


### d4

In [19]:
dict_category = {'category_emotion':[],'artist_disp_nm':[],'track_title':[],'album_style':[]}

for name_categorytheme,url_categorytheme in d4.items():
    soup_category = BeautifulSoup(requests.get(url_categorytheme).content, 'lxml')
    page_num = int(re.sub(r'\D', '', soup_category.select_one('p.desc').get_text()))//24 + 2
    base_url = soup_category.select('div.paging > a')[0]['href'][:-1]
    album_count=0
    for n_th_page in range(1,page_num): # page_num
        url_categorypages=base_url+f'{n_th_page}'
        dict_categoryalbum={}
        soup_pages = BeautifulSoup(requests.get(url_categorypages).content, 'lxml')
        for album_tag in soup_pages.find_all('a', class_="title hyrend"):
            dict_categoryalbum[album_tag['title']]=album_tag['href']
        for url_categoryalbum in dict_categoryalbum.values():
            album_count+=1
            soup_album = BeautifulSoup(requests.get(url_categoryalbum).content, 'lxml')
            ele_album_style = str() 
            for a_tag in soup_album.select('div.tag > a.hyrend'):
                ele_album_style+=a_tag.get_text()
            for music_tag in soup_album.find_all('a',attrs={'class': 'btnActions','album_title':False}): # soup_album.select('a.btnActions')
                dict_category['category_emotion'].append(name_categorytheme)
                dict_category['artist_disp_nm'].append(music_tag['artist_disp_nm'])
                dict_category['track_title'].append(music_tag['track_title'])
                dict_category['album_style'].append(ele_album_style)
        if n_th_page%20==0:
            print(f'{n_th_page}번째 페이지',f'앨범{album_count}개')
            display(pd.DataFrame.from_dict(dict_category).tail(2))
pd.DataFrame.from_dict(dict_category).shape
data = pd.DataFrame.from_dict(dict_category)
data.to_csv('data_emotion_d4.csv', index = False, encoding = 'UTF-8')
display(pd.DataFrame.from_dict(dict_category))

,category_emotion,artist_disp_nm,track_title,album_style
0,고백,뜨거운 감자(Hot Potato),고백,#국내#가요#고백#달달한
1,고백,더하얀(이지현),고백 연습 (feat. No Noo),#국내#가요#고백#달달한
2,고백,그리즐리 (Grizzly),그래서 그랬지,#국내#가요#고백#달달한
3,고백,견우 (Kyunwoo),관심병 (Feat. 버벌진트),#국내#가요#고백#달달한
4,고백,마이티 마우스(Mighty Mouth),나쁜놈 (feat. 소야),#국내#가요#고백#달달한
...,...,...,...,...
16406,새벽감성,노을,사랑이라면,#가요#잔잔한#국내#노래가사#새벽감성
16407,새벽감성,성시경,노래가 되어,#가요#잔잔한#국내#노래가사#새벽감성
16408,새벽감성,에피톤 프로젝트(Epitone Project),선인장 (Song By 심규선),#가요#잔잔한#국내#노래가사#새벽감성
16409,새벽감성,조장혁,In My Dream,#가요#잔잔한#국내#노래가사#새벽감성


### d5

In [20]:
dict_category = {'category_emotion':[],'artist_disp_nm':[],'track_title':[],'album_style':[]}

for name_categorytheme,url_categorytheme in d5.items():
    soup_category = BeautifulSoup(requests.get(url_categorytheme).content, 'lxml')
    page_num = int(re.sub(r'\D', '', soup_category.select_one('p.desc').get_text()))//24 + 2
    base_url = soup_category.select('div.paging > a')[0]['href'][:-1]
    album_count=0
    for n_th_page in range(1,page_num): # page_num
        url_categorypages=base_url+f'{n_th_page}'
        dict_categoryalbum={}
        soup_pages = BeautifulSoup(requests.get(url_categorypages).content, 'lxml')
        for album_tag in soup_pages.find_all('a', class_="title hyrend"):
            dict_categoryalbum[album_tag['title']]=album_tag['href']
        for url_categoryalbum in dict_categoryalbum.values():
            album_count+=1
            soup_album = BeautifulSoup(requests.get(url_categoryalbum).content, 'lxml')
            ele_album_style = str() 
            for a_tag in soup_album.select('div.tag > a.hyrend'):
                ele_album_style+=a_tag.get_text()
            for music_tag in soup_album.find_all('a',attrs={'class': 'btnActions','album_title':False}): # soup_album.select('a.btnActions')
                dict_category['category_emotion'].append(name_categorytheme)
                dict_category['artist_disp_nm'].append(music_tag['artist_disp_nm'])
                dict_category['track_title'].append(music_tag['track_title'])
                dict_category['album_style'].append(ele_album_style)
        if n_th_page%20==0:
            print(f'{n_th_page}번째 페이지',f'앨범{album_count}개')
            display(pd.DataFrame.from_dict(dict_category).tail(2))
pd.DataFrame.from_dict(dict_category).shape
data = pd.DataFrame.from_dict(dict_category)
data.to_csv('data_emotion_d5.csv', index = False, encoding = 'UTF-8')
display(pd.DataFrame.from_dict(dict_category))

20번째 페이지 앨범480개


,category_emotion,artist_disp_nm,track_title,album_style
23583,기분전환,Christiane,Get With You,#해외#해외 팝#팝송#일렉트로닉#일렉트로닉팝#편집숍/매장#리드미컬한#기분전환
23584,기분전환,Slowz,Nice And Slow (feat. Blush'ko),#해외#해외 팝#팝송#일렉트로닉#일렉트로닉팝#편집숍/매장#리드미컬한#기분전환


40번째 페이지 앨범960개


,category_emotion,artist_disp_nm,track_title,album_style
33149,기분전환,James Gang(제임스 갱),Walk Away,#해외#락#하드락/락켄롤#미국#드라이브#기분전환#산책/여행#상쾌한
33150,기분전환,The Who(더 후),Won't Get Fooled Again (Single Edit),#해외#락#하드락/락켄롤#미국#드라이브#기분전환#산책/여행#상쾌한


60번째 페이지 앨범1440개


,category_emotion,artist_disp_nm,track_title,album_style
43832,기분전환,Jay Sean(제이 션),I'm All Yours (feat. Pitbull),#해외#팝송#신나는#드라이브#기분전환#오후#출근/퇴근길#월요병#연휴병#해외 팝
43833,기분전환,Flo Rida(플로 라이다),My House,#해외#팝송#신나는#드라이브#기분전환#오후#출근/퇴근길#월요병#연휴병#해외 팝


20번째 페이지 앨범480개


,category_emotion,artist_disp_nm,track_title,album_style
54816,힐링,316,"다시, 첫눈 (Dear Snow, Thee Fall Again)",#연주곡#피아노#피아노연주곡#힐링#뉴에이지#휴식/명상#에스테틱#차분한#북카페#독서
54817,힐링,정재형,겨울의 정원,#연주곡#피아노#피아노연주곡#힐링#뉴에이지#휴식/명상#에스테틱#차분한#북카페#독서


,category_emotion,artist_disp_nm,track_title,album_style
0,싱숭생숭,Rosalyn,Loverfriend,#국내외#비/흐림#감성적인#밤/새벽#멜랑콜리#나른해지는#싱숭생숭#센치한
1,싱숭생숭,Bounty Island,Avocado Girl,#국내외#비/흐림#감성적인#밤/새벽#멜랑콜리#나른해지는#싱숭생숭#센치한
2,싱숭생숭,Mild High Club,Chasing My Tail,#국내외#비/흐림#감성적인#밤/새벽#멜랑콜리#나른해지는#싱숭생숭#센치한
3,싱숭생숭,Meego,sunset,#국내외#비/흐림#감성적인#밤/새벽#멜랑콜리#나른해지는#싱숭생숭#센치한
4,싱숭생숭,Strawberry Guy(스트로베리 가이),Mrs Magic,#국내외#비/흐림#감성적인#밤/새벽#멜랑콜리#나른해지는#싱숭생숭#센치한
...,...,...,...,...
56709,힐링,Celtic Woman(켈틱 우먼),"You Raise Me Up (Performed By Lisa Kelly, Mair...",#클래식#힐링#지치고 힘들때#클래식한
56710,힐링,Libera(리베라),Love And Mercy,#클래식#힐링#지치고 힘들때#클래식한
56711,힐링,The O\'Neill Brothers(오닐 브라더스),"Jesu, Joy Of Man's Desiring",#클래식#힐링#지치고 힘들때#클래식한
56712,힐링,Les Petits Chanteurs A La Croix De Bais(파리 나무 ...,제바에르: 우리 마을의 천사들 (Les Anges Dans Nos Campagnes),#클래식#힐링#지치고 힘들때#클래식한
